In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import torch
import open_clip
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/Benchmarking/Beetlepalooza_beetles_image_only.csv"
df = pd.read_csv(file_path, sep="\t")
df = df[['ImageFilePath', 'ScientificName']]
df = df.sample(n=1000, random_state=99)
df.head(2)

,ImageFilePath,ScientificName
1921,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Synuchus impunctatus
4058,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Pterostichus lachrymosus


In [4]:
model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms("hf-hub:imageomics/bioclip")
tokenizer = open_clip.get_tokenizer("hf-hub:imageomics/bioclip")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

for param in model.parameters() :
    param.requires_grad = False
    

In [5]:
def extract_image_features(image_path):
    
    image = Image.open(image_path).convert("RGB")

    image_tensor = preprocess_val(image).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image_tensor)
        image_features /= image_features.norm(dim=-1, keepdim=True)

    return image_features.cpu().numpy()


In [6]:
X = np.vstack([extract_image_features(img) for img in tqdm(df["ImageFilePath"], desc="Processing ...") ])

print(f"Extracted feature shape: {X.shape}")

le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()


Processing ...: 100%|██████████| 1000/1000 [01:18<00:00, 12.76it/s]

Extracted feature shape: (1000, 512)


In [10]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef


seed = 99


models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}


for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    predictions[name] = preds

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)

    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}

    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


NaiveBayes                | Acc: 71.50% | Prec: 65.46% | Rec: 71.50% | F1: 66.70% | Bal Acc: 42.71% | MCC: 0.6967
LogisticRegression        | Acc: 56.00% | Prec: 40.84% | Rec: 56.00% | F1: 44.94% | Bal Acc: 22.40% | MCC: 0.5307
NearestNeighbor           | Acc: 75.00% | Prec: 69.62% | Rec: 75.00% | F1: 70.69% | Bal Acc: 53.34% | MCC: 0.7346
MLP_Baseline              | Acc: 81.50% | Prec: 75.85% | Rec: 81.50% | F1: 77.80% | Bal Acc: 58.76% | MCC: 0.8038


In [11]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
116,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Pterostichus melanarius,Pterostichus melanarius,Pterostichus melanarius,Cyclotrachelus torvus,Cyclotrachelus torvus,Cyclotrachelus torvus,Pterostichus melanarius,Cyclotrachelus torvus,Cyclotrachelus torvus
6955,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Euryderus grossus,Euryderus grossus,Euryderus grossus,Euryderus grossus,Euryderus grossus,Euryderus grossus,Euryderus grossus,Euryderus grossus,Euryderus grossus


In [12]:
test_df.to_csv("/users/PAS2136/rayees/CV4A - MultiModality/Subset/BioCLIP.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/CV4A - MultiModality/Subset/BioCLIP-metrics.csv", index=False)